In [3]:
# modules
from container_maker_spec.service_pb2_grpc import ContainerMakerAPIStub
from container_maker_spec.types_pb2 import CreateContainerRequest, CreateContainerResponse
from container_maker_spec.types_pb2 import ManipulateContainerRequest
import src.utils as utils

# third party
import grpc

## Channels
The first step is to create GRPC channels. We have two types of channels, `insecure` and `secure`.
Based on the environment choose the right channel.

There are two cells, choose one of them.

In [12]:
# insecure channel
ic: grpc.Channel = grpc.insecure_channel("0.0.0.0:50052")
stub: ContainerMakerAPIStub = ContainerMakerAPIStub(channel=ic)

In [19]:
# secure channel
server_key: bytes = utils.read_certs('SERVER_KEY', './cert/server.key')
server_cert: bytes = utils.read_certs('SERVER_CRT', './cert/server.crt')
ca_cert: bytes = utils.read_certs('CA_CRT', './cert/ca.crt')

credentials: grpc.ServerCredentials = grpc.ssl_channel_credentials(
    root_certificates=ca_cert,
    private_key=client_key,
    certificate_chain=client_cert
)

sc = grpc.secure_channel("localhost:50052", credentials)
stub = ContainerMakerAPIStub(channel=sc)


## Operations
These are the operations:
1. Create Container
2. Start Container
3. Stop Container
4. Delete Container

Below we have examples for each

In [20]:
# Create Container
create_container_request: CreateContainerRequest = CreateContainerRequest(
    image_name="zim95/ssh_ubuntu:latest",
    container_name="test_grpc_ssh_ubuntu",
    container_network="test_grpc_ssh_ubuntu_network",
    publish_information={"22/tcp": "2222"},
    environment={"SSH_USERNAME": "zim95", "SSH_PASSWORD": "1234"}
)

response = stub.createContainer(create_container_request)
print(response.create_container_response_item)

[container_id: "test_id"
container_image: "zim95/ssh_ubuntu:latest"
container_name: "test_grpc_ssh_ubuntu"
container_network: "test_grpc_ssh_ubuntu_network"
container_port: 5
]


In [17]:
# Start Container
start_container_request: ManipulateContainerRequest = ManipulateContainerRequest(
    container_ids=["test_id"],
    container_network="test_grpc_ssh_ubuntu_network"
)
try:
    response = stub.startContainer(start_container_request)
    print(response)
except Exception as e:
    print(e.details())

Exception calling application: Test Exception
